In [1]:
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import os
import pandas as pd

from skimage import morphology, measure
from skimage.segmentation import expand_labels, watershed, relabel_sequential
from sklearn.preprocessing import minmax_scale
from scipy import ndimage as ndi
from dask_ml.cluster import KMeans
from dask_ml.decomposition import PCA

from dask_image import imread
from dask_image import ndmeasure
from dask_image import ndfilters
from dask import array as da
from dask import delayed
from dask.distributed import Client, progress
from colorsys import hls_to_rgb

import napari

import trimesh
from trimesh.curvature import (
    discrete_gaussian_curvature_measure,
    discrete_mean_curvature_measure,
    sphere_ball_intersection,
)

In [2]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=1, memory_limit='12GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.19.112.205:8787/status,
Dashboard: http://172.19.112.205:8787/status,Workers: 1
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://172.19.112.205/21916/1,Workers: 1
Dashboard: http://172.19.112.205:8787/status,Total threads: 6
Started: Just now,Total memory: 11.18 GiB
Comm: inproc://172.19.112.205/21916/4,Total threads: 6
Dashboard: http://172.19.112.205:61771/status,Memory: 11.18 GiB
Nanny: None,


In [3]:
#Specific is the name of the folder of the orginal data
specific = 'F231006L'
#which_finger should be 1,2,3, or 4, corresponding to the index, middle, ring, and pinky fingers
finger_num = 1
digit_num = 2

In [4]:
finger_names = {1: 'index', 2: 'middle', 3: 'ring', 4: 'pinky'}
finger_name = finger_names[finger_num]
digit_names = {1: 'proximal', 2: 'middle', 3: 'distal'}
digit_name = digit_names[digit_num]

In [5]:
full_finger_dir = f'../fingers/{finger_name}/full/{specific}/'
digit_dir = f'../fingers/{finger_name}/{digit_name}/{specific}/'
finger_eig_dir = f'../fingers/{finger_name}/full/{specific} - eigens/'
mesh_dir = f'../meshes_npy/{finger_name}/{digit_name}/{specific}/'
mesh_csv_dir = f'../mesh_csvs/{finger_name}/{digit_name}/{specific}/'
digit_eig_dir = f'../fingers/{finger_name}/{digit_name}/{specific} - eigens/'
spacing = np.load('../spacing/' + specific + '.npy')

In [6]:
vertices = np.load(mesh_dir + 'vertices.npy')
faces = np.load(mesh_dir + 'faces.npy')
values = np.load(mesh_dir + 'values.npy')
normals = np.load(mesh_dir + 'normals.npy')

In [7]:
vertices

array([[ 2.14186178, 19.4612018 ,  9.7679398 ],
       [ 2.1794383 , 19.4612018 ,  9.7303708 ],
       [ 2.1794383 , 19.42363191,  9.7679398 ],
       ...,
       [14.84272638,  4.50838652,  9.31711181],
       [14.84272638,  4.58352629,  9.24197381],
       [14.84272638,  4.58352629,  9.31711181]])

In [8]:
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

In [9]:
mesh

<trimesh.Trimesh(vertices.shape=(164976, 3), faces.shape=(329948, 3))>

In [10]:
mesh.is_watertight

True

In [11]:
mesh.euler_number

2

In [12]:
defects = trimesh.curvature.vertex_defects(mesh)

In [13]:
defects

array([ 7.85562919e-01, -1.33226763e-14,  5.50670620e-14, ...,
        0.00000000e+00,  2.61855363e-01,  5.23707556e-01])

In [14]:
o_d = trimesh.convex.hull_points(mesh)

In [15]:
o_d

TrackedArray([[ 2.14186178, 19.4612018 ,  9.7679398 ],
              [ 2.14186178, 19.53634157,  9.8430778 ],
              [ 2.14186178, 19.68662112,  9.6176638 ],
              ...,
              [14.84272638,  4.50838652,  9.16683581],
              [14.84272638,  4.58352629,  9.24197381],
              [14.84272638,  4.58352629,  9.31711181]])

In [16]:
viewer = napari.Viewer()

In [17]:
mesh_view = viewer.add_surface(
    data = (vertices, faces, values),
)

In [ ]:
convex_points = viewer.add_points(
    o_d,
    size = 1,
)

In [ ]:
v_z = vertices @ np.array([0,0,1])

In [ ]:
centroid = np.average(vertices, axis=0)
centroid

In [ ]:
p_1_i = np.argmin(v_z)
p_1 = vertices[p_1_i]
p_1

In [ ]:
n = np.array([0,0,1])

In [ ]:
verts_c_p_1 = vertices[((vertices - centroid) @ (p_1 - centroid))/(np.linalg.norm(p_1 - centroid)) > 1] - p_1

In [ ]:
r = np.abs((verts_c_p_1) @ n) * np.linalg.norm(verts_c_p_1, axis = 1)**(-1)
r

In [ ]:
p_2_i = np.argmax(r)
if p_2_i >= p_1_i:
    p_2_i += 1
p_2_i

In [ ]:
bottom_point = viewer.add_points(
    p_1,
    size = 1,
)